# --------Extract Salaries using scraping--------

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [3]:
company_name = "Apple"
job_title = "Software engineer"
location = "India"
# add headless option
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(options = options)
# url = f"https://www.glassdoor.co.in/Salary/Google-Software-Engineer-India-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"
# https://www.glassdoor.co.in/Salary/Apple-Software-Engineer-Salaries-E1138_D_KO6,23.htm
url = f"https://www.glassdoor.co.in/Salary/{company_name}-{job_title.replace(' ', '-')}-{location}-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"

driver.get(url)
driver.maximize_window()
# wait for the page to load completely
time.sleep(2)
# collect raw html
response = driver.page_source
# driver.close()

## Now to parse and scan the html tree

In [9]:
import re
from html.parser import HTMLParser

class customizedParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.result = []
    def handle_data(self, data):
        self.result.append(data)
        
def do():
    company_name = "Google"
    job_title = "Software engineer"
    location = "India"
    # add headless option
    options = Options()
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options = options)
    # url = f"https://www.glassdoor.co.in/Salary/Google-Software-Engineer-India-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"
    url = f"https://www.glassdoor.co.in/Salary/{company_name}-{job_title.replace(' ', '-')}-{location}-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"
    
    driver.get(url)
    driver.maximize_window()
    # wait for the page to load completely
    time.sleep(2)
    # collect raw html
    response = driver.page_source
    # driver.close()
    soup = BeautifulSoup(response, 'html.parser')
    driver.close()
    # Locate the section
    section = soup.find("section", class_ = "hero_SubModule__wCYtE")
    # if the section is not present
    if not section:
        return {
            "error" : "No salary range defined"
        }
    div = section.find('div', class_ = 'hero_TotalPayLayout__X55hl hero_PayRange__nKzVj')
    div_salary_range = div.find_all('div', 'hero_PayRange__nKzVj')
    # print(div_salary_range) # we got a list like
    '''
[<div class="hero_PayRange__nKzVj">₹15.0L</div>, <div class="hero_PayRange__nKzVj">-</div>, 
<div class="hero_PayRange__nKzVj">₹35.0L<sub class="hero_PayLabel__UtI9Y">/yr</sub></div>]
    '''
    salary = ""
    for unparsed_data in div_salary_range:
         # instantiating our customized HTM parser
        parser = customizedParser()
        # feed the unparsed data to parser
        parser.feed(repr(unparsed_data))
        # get the resultant list
        salary += ' '.join(parser.result)
    return salary

In [3]:
print(do())

₹15.0L-₹35.0L /yr


In [45]:
from html.parser import HTMLParser

class TextExtractor(HTMLParser):
    def __init__(self):
        super().__init__()
        self.result = []

    def handle_data(self, data):
        self.result.append(data.strip())

html = '<div class="hero_PayRange__nKzVj">₹35.0L<sub class="hero_PayLabel__UtI9Y">/yr</sub></div>'

parser = TextExtractor()
parser.feed(html)
output = ''.join(parser.result)
print(output)

₹35.0L/yr


## **HTML Parser library**
```html.parser -> feed()```

In [47]:
from html.parser import HTMLParser
# We need to customize the parser or else it contains blank placeholders
class customizedParser(HTMLParser):
    def __init__(self):
        # Call parent class constructor
        super().__init__()
        # Container to hold data
        self.result = []
    # customized the way to handle data
    def handle_data(self, data):
        self.result.append(data.strip())

# __main__ segment
' instantiate customized html parser '
myParser = customizedParser()
unparsed = '<div class="hero_PayRange__nKzVj">₹35.0L<sub class="hero_PayLabel__UtI9Y">/yr</sub></div>'
# use feed function
myParser.feed(unparsed)
parsed = myParser.result
print(parsed)

['₹35.0L', '/yr']


# **Using levels.fyi**
Let's try to open levels.fyi
Sample link - ```https://www.levels.fyi/companies/google/salaries/software-engineer?country=254```

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time

In [2]:
def do(company, job):
    options = Options()
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options = options)
    url = f"https://www.levels.fyi/companies/{company}/salaries/{job.replace(' ','-')}"
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)
    # get the raw html
    response = driver.page_source
    soup = BeautifulSoup(response, 'html.parser')
    # driver.close()
    # get the table
    table = soup.find('table', class_ = 'MuiTable-root css-1f6fkxk')
    if not table:
        return None
    print('table found')
    result = []
    # find tbody
    tbody = table.find('tbody', class_ = 'MuiTableBody-root job-family_tableBody__MaiIw css-1xnox0e')
    if not tbody:
        return None
    print('found tbody')
    # find all rows
    rows = tbody.find_all('tr', class_ = 'MuiTableRow-root job-family_bodyRow__WuAug css-140sacz')
    # traver each row
    for row in rows:
        # get columns as td value
        col = row.find_all('td', class_ = 'MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-sizeMedium css-k1mugk')
        # we want minimum results
        if len(col) >= 5:
            salary_record = {
                'level': col[0].get_text(strip = True),
                'Total': col[1].get_text(strip = True),
                'Base': col[2].get_text(strip = True),
                'Stock (/yr)': col[3].get_text(strip = True),
                'Bonus': col[4].get_text(strip = True)
            }
            result.append(salary_record)
    # return result

In [13]:
company = input('Enter company name : ')
job = input('Enter job : ')
print(do(company, job))

Enter company name :  apple
Enter job :  software engineer


table found
found tbody
None


### **We will also be expanding results using xpath**

In [6]:
def do(company, job):
    options = Options()
    options.add_argument("--disable-gpu")
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options = options)
    url = f"https://www.levels.fyi/companies/{company}/salaries/{job.replace(' ','-')}/locations/new-york"
    driver.get(url)
    driver.maximize_window()
    
    try:
        # Wait for the clickable button
        wait = WebDriverWait(driver, 10)
        view_more_levels = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textNeutral MuiButton-sizeLarge MuiButton-textSizeLarge MuiButton-colorNeutral MuiButton-root MuiButton-text MuiButton-textNeutral MuiButton-sizeLarge MuiButton-textSizeLarge MuiButton-colorNeutral css-y5b368']")))
        view_more_levels.click()
        print('clicked sir')
        time.sleep(2)
    except Exception as e:
        print('button not found:', e)
    # get the raw html
    response = driver.page_source
    soup = BeautifulSoup(response, 'html.parser')
    driver.close()
    # get the table
    table = soup.find('table', class_ = 'MuiTable-root css-1f6fkxk')
    if not table:
        return None
    print('table found')
    result = []
    # find tbody
    tbody = table.find('tbody', class_ = 'MuiTableBody-root job-family_tableBody__MaiIw css-1xnox0e')
    if not tbody:
        return None
    print('found tbody')
    # find all rows
    rows = tbody.find_all('tr', class_ = 'MuiTableRow-root job-family_bodyRow__WuAug css-140sacz')
    # traver each row
    for row in rows:
        # get columns as td value
        col = row.find_all('td', class_ = 'MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-sizeMedium css-k1mugk')
        # we want minimum results
        if len(col) >= 5:
            salary_record = {
                'level': col[0].get_text(strip = True),
                'Total': col[1].get_text(strip = True),
                'Base': col[2].get_text(strip = True),
                'Stock (/yr)': col[3].get_text(strip = True),
                'Bonus': col[4].get_text(strip = True)
            }
            result.append(salary_record)
    return result

In [7]:
print(do('microsoft', 'software engineer'))

button not found: Message: 
Stacktrace:
#0 0x5abccc06bc4a <unknown>
#1 0x5abccbb116e0 <unknown>
#2 0x5abccbb63117 <unknown>
#3 0x5abccbb63311 <unknown>
#4 0x5abccbbb1ec4 <unknown>
#5 0x5abccbb88e5d <unknown>
#6 0x5abccbbaf2cc <unknown>
#7 0x5abccbb88c03 <unknown>
#8 0x5abccbb5547b <unknown>
#9 0x5abccbb560e1 <unknown>
#10 0x5abccc0303fb <unknown>
#11 0x5abccc03432f <unknown>
#12 0x5abccc017f39 <unknown>
#13 0x5abccc034ec8 <unknown>
#14 0x5abccbffc68f <unknown>
#15 0x5abccc0592b8 <unknown>
#16 0x5abccc059496 <unknown>
#17 0x5abccc06ab26 <unknown>
#18 0x78e4f049caa4 <unknown>
#19 0x78e4f0529c3c <unknown>

None


In [6]:
import http.client
import urllib.parse
from pprint import pprint
import json
conn = http.client.HTTPSConnection("jsearch.p.rapidapi.com")

### _**Starting with Jsearch (Rapid API)**_
<font color = 'red'>200 requests/month</font><br>
[Note]: We will be using urllib.parse module to automatically encodes special characters and spaces like "nodejs developer" -> nodejs%20developer

In [7]:
url = "jsearch.p.rapidapi.com"

In [8]:
headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "jsearch.p.rapidapi.com"
}

```conn.request("GET", "/company-job-salary?company=Amazon&job_title=software%20developer&location_type=ANY&years_of_experience=ALL", headers=headers)```

In [9]:
parameters = {
    'company': 'Apple',
    'job_title': 'software engineer',
    'location' : 'New York',
    'location_type': 'ANY',
    'years_of_experience': 'ALL',
}

Now, encode the parameters into a query string such that it is url-safe

In [10]:
url_safe_query = urllib.parse.urlencode(parameters)
print(url_safe_query)

company=Apple&job_title=software+engineer&location=New+York&location_type=ANY&years_of_experience=ALL


In [11]:
import json
conn.request("GET", f"/company-job-salary?{url_safe_query}", headers=headers)
# conn.request("GET", f"/estimated-salary?{url_safe_query}", headers=headers)
res = conn.getresponse()
data = res.read()
type(data)
print(data)

b'{"status":"OK","request_id":"4422f117-55cf-4ae6-b9c2-fb636b8927b9","data":[{"location":"New York City, NY","job_title":"Software Engineer","company":"Apple","min_salary":228183.85,"max_salary":374313.83,"median_salary":289326.03,"min_base_salary":149955.72,"max_base_salary":228287.99,"median_base_salary":185021.86,"min_additional_pay":78228.13,"max_additional_pay":146025.84,"median_additional_pay":104304.17,"salary_period":"YEAR","salary_currency":"USD","confidence":"CONFIDENT","salary_count":303}]}'


In [12]:
output_dict = json.loads(data)
type(output_dict)

dict

In [13]:
from pprint import pprint

In [14]:
pprint(output_dict['data'][0])

{'company': 'Apple',
 'confidence': 'CONFIDENT',
 'job_title': 'Software Engineer',
 'location': 'New York City, NY',
 'max_additional_pay': 146025.84,
 'max_base_salary': 228287.99,
 'max_salary': 374313.83,
 'median_additional_pay': 104304.17,
 'median_base_salary': 185021.86,
 'median_salary': 289326.03,
 'min_additional_pay': 78228.13,
 'min_base_salary': 149955.72,
 'min_salary': 228183.85,
 'salary_count': 303,
 'salary_currency': 'USD',
 'salary_period': 'YEAR'}


### _**Starting with CareerJet API**_
<font color = 'red'>1000 requests per hour.</font>

In [21]:
import requests

url = "http://public.api.careerjet.net/search"

params = {
    "locale_code": "en_US",
    "keywords": "software engineer Apple",
    "location": "new york",
    "affid": "09a7e99d93482b93310a29944476cd33",  
    "user_ip": "1.2.3.4",        
    "user_agent": "Mozilla/5.0", 
    "pagesize": 10,              
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

In [22]:
data

{'jobs': [{'salary_min': '90000',
   'locations': 'Hudson Falls, NY',
   'salary_type': 'Y',
   'date': 'Mon, 05 May 2025 07:13:14 GMT',
   'description': ':   As an Infrastructure <b>Engineer</b>, you will be expected to demonstrate professional expertise in designing, building, and supporting enterprise... and recommend network hardware, systems management <b>software</b>, and architecture.   4. Build, configure, and maintain network switches...',
   'salary_currency_code': 'USD',
   'salary': '$90000 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1818416443021d0f540052160b151b15000031060e1a444e585f2f2101094e1a45061a6327090352151b1d015f4849585f016e5f11411755/3460c4da95e3071946d20ca342490d35.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Infrastructure Engineer (Req #: 1061)',
   'salary_max': '90000',
   'company': 'Peckham Industries'},
  {'locations': 'New York City, NY',
   'site': '',
   'date': 'Mon, 19 May 2025 02:32:21 GMT',
   'url': 'http

In [3]:
def transform_job_listings(data):
    jobs = data.get('jobs', [])
    if not jobs:
        return {}

    company = jobs[0].get('company', None)
    location = jobs[0].get('locations', None)
    currency = jobs[0].get('salary_currency_code', None)
    salary_type = jobs[0].get('salary_type', None)
    job_title = 'Software Engineer'  # Normalize for this dataset

    # Extract base salaries (assumes salary_min == salary_max for each job)
    base_salaries = []
    for job in jobs:
        try:
            salary = float(job.get('salary_min') or 0)
            base_salaries.append(salary)
        except ValueError:
            continue

    if not base_salaries:
        return {}

    min_base = min(base_salaries)
    max_base = max(base_salaries)
    median_base = sorted(base_salaries)[len(base_salaries) // 2]

    return {
        'company': company,
        # 'confidence': 'CONFIDENT',  # Assumed
        'job_title': job_title,
        'location': location,
        # 'max_additional_pay': None,  # Not provided in data
        'max_base_salary': max_base,
        # 'max_salary': max_base,
        # 'median_additional_pay': None,
        'median_base_salary': median_base,
        # 'median_salary': median_base,
        # 'min_additional_pay': None,
        # 'min_base_salary': min_base,
        # 'min_salary': min_base,
        # 'salary_count': len(base_salaries),
        'salary_currency': currency,
        'salary_period': 'YEAR' if salary_type == 'Y' else None
    }

Since the dataset does not include additional pay (e.g., bonuses, stock), we'll assume:

    additional_pay is 0 for all listings (and therefore min_salary = base, max_salary = base, etc.).

    confidence is assumed CONFIDENT since it's a large number of listings from a reputable source.

    job_title is normalized as Software Engineer.

In [7]:
# type(data)
data
# transform_job_listings(data)

{'jobs': [{'salary_min': '151300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Wed, 21 May 2025 06:31:35 GMT',
   'description': '. This is an opportunity to make a significant impact on the future of the <b>Amazon</b> vision.   As a Senior <b>Software</b> <b>Engineer</b>, you will drive the...DESCRIPTION   <b>Amazon</b> is investing heavily in building a world class advertising business and we are responsible...',
   'salary_currency_code': 'USD',
   'salary': '$151300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-194d417e480a06015554730c0e1519061745542d0714434558485f663c01410057021a044e220b471d060c1658296e48430d001c07314e04010f0b021722270d071a45591d7e42021b1946064561290c0f1d0a4e542d0714434558485f655e58104c13/d444ebe9994582edd3a15b3fe94820b2.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Senior Software Engineer (SDEIII), Amazon Ads',
   'salary_max': '151300',
   'company': 'Amazon'},
  {'salary_min': '151300',
   'loc

### _**Glass door real time API** (Unofficial API)_
<font color = 'red'>200 requests/month</font>

In [15]:
import http.client
import urllib.parse

In [2]:
# define the connection
conn = http.client.HTTPSConnection("glassdoor-real-time.p.rapidapi.com")

In [3]:
# define the headers
headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "glassdoor-real-time.p.rapidapi.com"
}

**First of all we need to work on locationId**

In [4]:
params = {
    'query' : 'new york'
}
url_safe_query = urllib.parse.urlencode(params)
print(url_safe_query)

query=new+york


In [5]:
conn.request("GET", f"/jobs/location?{url_safe_query}", headers=headers)

In [6]:
import json
res = conn.getresponse()
data = res.read()
output_dict = json.loads(data)
print(output_dict)

{'data': [{'countryId': 1, 'locationId': 'eyJ0IjoiQyIsImlkIjoxMTMyMzQ4LCJuIjoiTmV3IFlvcmssIE5ZIn0=', 'locationName': 'New York, NY', 'locationType': 'C'}, {'countryId': 1, 'locationId': 'eyJ0IjoiUyIsImlkIjo0MjgsIm4iOiJOZXcgWW9yayBTdGF0ZSJ9', 'locationName': 'New York State', 'locationType': 'S'}, {'countryId': 1, 'locationId': 'eyJ0IjoiQyIsImlkIjoxMTMyOTEzLCJuIjoiTmV3IFlvcmsgTWlsbHMsIE5ZIn0=', 'locationName': 'New York Mills, NY', 'locationType': 'C'}, {'countryId': 1, 'locationId': 'eyJ0IjoiQyIsImlkIjoxMTQyNDI0LCJuIjoiTmV3IFlvcmsgTWlsbHMsIE1OIn0=', 'locationName': 'New York Mills, MN', 'locationType': 'C'}, {'countryId': 1, 'locationId': 'eyJ0IjoiQyIsImlkIjozNzMyMDk3LCJuIjoiTmV3IFlvcmssIE1PIn0=', 'locationName': 'New York, MO', 'locationType': 'C'}, {'countryId': 1, 'locationId': 'eyJ0IjoiQyIsImlkIjozODE3Nzc3LCJuIjoiTmV3IFlvcmssIEZMIn0=', 'locationName': 'New York, FL', 'locationType': 'C'}, {'countryId': 1, 'locationId': 'eyJ0IjoiQyIsImlkIjo1MDIyMTM1LCJuIjoiTWlkdG93biBOZXcgWW9yaywgTl

```output_dict['data'][0]['locationId']```

In [7]:
output_dict['data'][0]['locationId']

'eyJ0IjoiQyIsImlkIjoxMTMyMzQ4LCJuIjoiTmV3IFlvcmssIE5ZIn0='

**Now let us focus on getting the salary data**

In [8]:
params = {
    'query' : 'microsoft software engineer',
    'locationId': output_dict['data'][0]['locationId'],
    'limit': 10,
    'page': 1,
    'sort' : 'POPULAR'
}

In [9]:
url_safe_query = urllib.parse.urlencode(params)
print(url_safe_query)

query=microsoft+software+engineer&locationId=eyJ0IjoiQyIsImlkIjoxMTMyMzQ4LCJuIjoiTmV3IFlvcmssIE5ZIn0%3D&limit=10&page=1&sort=POPULAR


In [10]:
conn.request("GET", f"/salaries/search?{url_safe_query}", headers=headers)

In [11]:
res = conn.getresponse()
data = res.read()
print(data.decode('utf-8'))

{"data":{"aggregateSalaryResponse":{"numPages":105,"queryLocation":{"id":1132348,"name":"New York, NY","type":"CITY"},"resultCount":1041,"results":[{"basePayStatistics":{"mean":160980.7},"currency":{"code":"USD","id":1},"employer":{"counts":{"globalJobCount":{"jobCount":742}},"id":1651,"name":"Microsoft Corporation","ratings":{"overallRating":4.2},"shortName":"Microsoft","squareLogoUrl":"https://media.glassdoor.com/sql/1651/microsoft-squarelogo-1479856042252.png"},"jobTitle":{"gocId":100063,"id":4279,"text":"Software Engineer"},"payPeriod":"ANNUAL","totalAdditionalPayStatistics":{"mean":57123.0},"totalPayStatistics":{"__typename":"StatisticsResult","percentiles":[{"ident":"P10","value":151869.73},{"ident":"P25","value":179986.26},{"ident":"P50","value":218103.7},{"ident":"P75","value":268932.58},{"ident":"P90","value":322786.2}]},"salaryId":"eyJjaWQiOjExMzIzNDgsInR5cGUiOiJDSVRZIiwiZUlkIjoxNjUxLCJuIjoiTWljcm9zb2Z0IiwianQiOiJTb2Z0d2FyZSBFbmdpbmVlciIsImppIjo0Mjc5LCJnSSI6MTAwMDYzfQ=="},{"b

### **Transform to valuable dictionary**

In [39]:
def transform_salary_data(full_data):
    results = full_data['data']['aggregateSalaryResponse']['results']
    location = full_data['data']['aggregateSalaryResponse']['queryLocation']['name']
    salary_count = full_data['data']['aggregateSalaryResponse']['resultCount']
    
    output = []
    
    for job in results:
        job_title = job['jobTitle']['text']
        employer = job['employer']['name']
        currency = job['currency']['code']
        pay_period = job['payPeriod']

        base = job['basePayStatistics']['mean']
        additional = job['totalAdditionalPayStatistics']['mean']
        total_percentiles = {p['ident']: p['value'] for p in job['totalPayStatistics']['percentiles']}

        record = {
            'company': employer,
            'confidence': 'CONFIDENT',  # Assuming high confidence from Glassdoor data
            'job_title': job_title,
            'location': location,
            'min_base_salary': round(total_percentiles['P10'] - additional, 2),
            'median_base_salary': round(total_percentiles['P50'] - additional, 2),
            'max_base_salary': round(total_percentiles['P90'] - additional, 2),
            'min_additional_pay': round(additional * 0.85, 2),  # Approximation
            'median_additional_pay': round(additional, 2),
            'max_additional_pay': round(additional * 1.3, 2),   # Approximation
            'min_salary': round(total_percentiles['P10'], 2),
            'median_salary': round(total_percentiles['P50'], 2),
            'max_salary': round(total_percentiles['P90'], 2),
            'salary_count': salary_count,
            'salary_currency': currency,
            'salary_period': 'YEAR' if pay_period == 'ANNUAL' else pay_period
        }

        output.append(record)

    return output


In [40]:
def transform(entry, location_name):
    company = entry["employer"]["shortName"]
    job_title = entry["jobTitle"]["text"]
    currency = entry["currency"]["code"]
    pay_period = entry["payPeriod"].lower()
    rating = entry["employer"]["ratings"].get("overallRating", None)

    # Extract salary stats
    base_mean = entry["basePayStatistics"].get("mean", 0)
    base_median = entry["basePayStatistics"].get("median", 0)
    base_min = entry["basePayStatistics"].get("min", 0)
    base_max = entry["basePayStatistics"].get("max", 0)

    additional_mean = entry["totalAdditionalPayStatistics"].get("mean", 0)
    additional_median = entry["totalAdditionalPayStatistics"].get("median", 0)
    additional_min = entry["totalAdditionalPayStatistics"].get("min", 0)
    additional_max = entry["totalAdditionalPayStatistics"].get("max", 0)

    # Compute total salaries
    total_mean = entry["totalPayStatistics"].get("mean", base_mean + additional_mean)
    total_median = base_median + additional_median
    total_min = base_min + additional_min
    total_max = base_max + additional_max

    simplified = {
        "company": company,
        "job_title": job_title,
        "location": location_name,
        "salary_currency": currency,
        "salary_period": pay_period,
        "salary_range": f"{currency} {int(total_min):,} - {int(total_max):,}",
        "median_salary": f"{currency} {int(total_median):,}",
        "mean_salary": f"{currency} {int(total_mean):,}"
    }

    if rating:
        simplified["company_rating"] = rating

    return simplified


In [62]:
# output_dict = json.loads(data)
type(data)
type(res)
# output_dict = transform(output_dict, 'new york')
# print(output_dict)
# data = output_dict["data"]["aggregateSalaryResponse"]
# location_name = data["queryLocation"]["name"]
# entry = data["results"][0]
# ans = transform(entry, location_name)
# print(ans)

http.client.HTTPResponse

In [46]:
## **GlassDoor V2**

In [16]:
def glassDoor__v2(company, job_title, location):
    conn = http.client.HTTPSConnection("real-time-glassdoor-data.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
        'x-rapidapi-host': "real-time-glassdoor-data.p.rapidapi.com"
    }
    # first get the company ID --> [1st API call]
    query_params = {
        'query': company,
        'limit': 1,
        'domain': 'www.glassdoor.com'
    }
    url_safe_query = urllib.parse.urlencode(query_params)
    # make a request to get company_id
    conn.request("GET", f"/company-search?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read() # byte-string
    output_dict = json.loads(data) # dict-format
    # go to the data section
    data = output_dict.get('data',{})
    company_id = data[0].get('company_id',None)

    # Now start for salary-info search ---> [2nd API Call]
    query_params = {
        'company_id': company_id,
        'job_title': job_title,
        'location': location,
        'location_type' : 'ANY',
        'years_of_experience' : 'ALL',
        'domain': 'www.glassdoor.com'
    }
    url_safe_query = urllib.parse.urlencode(query_params)
    # request to get salary data
    conn.request("GET", f"/company-salaries?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    output_dict = json.loads(data) # dict-format
    return output_dict
    # convert it into structured format
    # def getStructuredFormat(output_dict):
    #     # get the parameters section
    #     parameters = output_dict.get('parameters',{})
    #     # get the data section
    #     data = output_dict.get('data',{})
    #     # get the required fields
    #     currency = data.get('salary_currency', None)
    #     min_sal = data.get('min_salary', 0.0)
    #     max_sal = data.get('max_salary', 0.0)

    #     format = {
    #         'company_name': data.get('company_name', None),
    #         'job_title': parameters.get('job_title', None),
    #         'location': parameters.get('location', None),
    #         # base salary range
    #         'min_base_salary': data.get('min_base_salary', None),
    #         'median_base_salary': data.get('median_base_salary', None),
    #         'max_base_salary': data.get('max_base_salary', None),
    #         # general salary range
    #         'min_salary': min_sal,
    #         'median_salary': data.get('median_salary', None),
    #         # median-additional-pay = mean-salary
    #         'mean_salary': data.get('median_additional_pay', None),
    #         'max_salary': max_sal,
    #         'salary_period': data.get('salary_period', None),
    #         'salary_range': f'{currency} {min_sal} - {currency} {max_sal}',
    #         'salary_currency': data.get('salary_currency', None),
    #     }
    #     return format
    # return getStructuredFormat(output_dict)

In [17]:
ans = glassDoor__v2('apple', 'software engineer', 'new york')

In [18]:
print(ans)

{'status': 'OK', 'request_id': '0a684582-1939-4909-8052-5ed59b7d1bf3', 'parameters': {'job_title': 'software engineer', 'company_id': 1138, 'domain': 'www.glassdoor.com', 'location': 'new york', 'location_type': 'ANY', 'years_of_experience': None}, 'data': {'location': 'New York City, NY', 'job_title': 'Software Engineer', 'company_id': 1138, 'company_name': 'Apple', 'min_salary': 228183.85, 'max_salary': 374313.83, 'median_salary': 289326.03, 'min_base_salary': 149955.72, 'max_base_salary': 228287.99, 'median_base_salary': 185021.86, 'min_additional_pay': 78228.13, 'max_additional_pay': 146025.84, 'median_additional_pay': 104304.17, 'salary_period': 'YEAR', 'salary_currency': 'USD', 'confidence': 'CONFIDENT', 'salary_count': 303}}


### _**Job Salary Data API**_
<font color = 'red'>(50 Requests/month)</font>

In [71]:
import http.client

conn = http.client.HTTPSConnection("job-salary-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "job-salary-data.p.rapidapi.com"
}

In [72]:
params = {
    'company': 'Amazon',
    'job_title': 'software developer',
    'location': 'new york',
    'location_type': 'ANY',
    'years_of_experience': 'ALL'
}

In [73]:
import urllib.parse
url_safe_query = urllib.parse.urlencode(params)
print(url_safe_query)

company=Amazon&job_title=software+developer&location=new+york&location_type=ANY&years_of_experience=ALL


In [74]:
conn.request("GET", f"/company-job-salary?{url_safe_query}", headers=headers)

In [46]:
def transform_estimate_data(data):
    item = data['data'][0]
    
    return {
        'company': item['company'],
        'confidence': item['confidence'],
        'job_title': item['job_title'],
        'location': item['location'],
        'min_base_salary': item['min_base_salary'],
        'median_base_salary': item['median_base_salary'],
        'max_base_salary': item['max_base_salary'],
        'min_additional_pay': item['min_additional_pay'],
        'median_additional_pay': item['median_additional_pay'],
        'max_additional_pay': item['max_additional_pay'],
        'min_salary': item['min_salary'],
        'median_salary': item['median_salary'],
        'max_salary': item['max_salary'],
        'salary_count': item['salary_count'],
        'salary_currency': item['salary_currency'],
        'salary_period': item['salary_period']
    }


In [75]:
res = conn.getresponse()
data = res.read()
ans = data.decode('utf-8')

In [76]:
output_dict = json.loads(data)
# output_dict = transform_estimate_data(output_dict)
print(output_dict)

{'status': 'OK', 'request_id': '92815ef3-ac3d-4a08-b64a-1c76b3a334f0', 'data': [{'location': 'New York City, NY', 'job_title': 'Software Developer', 'company': 'Amazon', 'min_salary': 177650.22, 'max_salary': 276253.84, 'median_salary': 219151.35, 'min_base_salary': 120605.66, 'max_base_salary': 169770.67, 'median_base_salary': 143091.94, 'min_additional_pay': 57044.56, 'max_additional_pay': 106483.17, 'median_additional_pay': 76059.41, 'salary_period': 'YEAR', 'salary_currency': 'USD', 'salary_count': 92, 'confidence': 'CONFIDENT', 'result': {'__typename': 'AggregatedSalaryEstimatesResult', 'basePayStatistics': {'__typename': 'StatisticsResult', 'max': 169770.67, 'mean': 143091.94, 'min': 120605.66, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 103418.91}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 120605.66}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 143091.94}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 169770.67}, {'__typena

## <font color = 'red'> Total requests - 200/MO + 200/MO + 50/MO + 1000/hr + Scraping from leves.fyi + Scraping from glassDoor(simplified range)</font>

**From chatGPT**

```
from functools import reduce

def extract_common_keys(dicts):
    """Get keys that are common across all dictionaries (top-level only)."""
    return set.intersection(*(set(d.keys()) for d in dicts))

def deep_merge_common(dicts):
    """Recursively extract and merge common keys and values from a list of dicts."""
    if not dicts:
        return {}

    common_keys = extract_common_keys(dicts)
    merged = {}

    for key in common_keys:
        # Check if the value is a nested dict in all cases
        if all(isinstance(d[key], dict) for d in dicts):
            merged[key] = deep_merge_common([d[key] for d in dicts])
        else:
            # If all values are equal, take one of them
            values = [d[key] for d in dicts]
            if all(v == values[0] for v in values):
                merged[key] = values[0]
            else:
                merged[key] = values  # keep all values if they differ

    return merged
```

In [63]:
def extract_salary_data(data: dict, company: str, location: str):
    results = data.get('data', {}).get('aggregateSalaryResponse', {}).get('results', [])
    company_rating = data.get('data', {}).get('employer', {}).get('overallRating', None)

    entries = []

    def fmt(salary):
        return f"USD {int(round(salary)):,}" if salary else "N/A"

    for job in results:
        job_title = job.get('jobTitle', {}).get('text', '')

        percentiles = job.get('totalPayStatistics', {}).get('percentiles', [])
        mean_salary = job.get('totalPayStatistics', {}).get('mean', None)

        p25 = next((p['value'] for p in percentiles if p.get('ident') == 'P25'), None)
        p50 = next((p['value'] for p in percentiles if p.get('ident') == 'P50'), None)
        p75 = next((p['value'] for p in percentiles if p.get('ident') == 'P75'), None)

        entry = {
            'company': company,
            'job_title': job_title,
            'location': location,
            'salary_currency': 'USD',
            'salary_period': 'annual',
            'salary_range': f"{fmt(p25)} - {fmt(p75)}" if p25 and p75 else "N/A",
            'median_salary': fmt(p50),
            'mean_salary': fmt(mean_salary),
            'company_rating': round(company_rating, 1) if company_rating else "N/A"
        }

        entries.append(entry)

    return entries


In [68]:
def glassDoor(company, job_title, location):
    conn = http.client.HTTPSConnection("glassdoor-real-time.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
        'x-rapidapi-host': "glassdoor-real-time.p.rapidapi.com"
    }
    # first extract locationId
    params = {
        'query' : location
    }
    url_safe_query = urllib.parse.urlencode(params)
    conn.request("GET", f"/jobs/location?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    res = json.loads(data)
    locationId = res['data'][0]['locationId'] # we got the location ID
    # Now extract salary data
    params = {
        'query' : f'{company} {job_title}',
        'locationId': locationId,
        'limit': 10,
        'page': 1,
        'sort' : 'POPULAR'
    }
    url_safe_query = urllib.parse.urlencode(params)
    conn.request("GET", f"/salaries/search?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    res = json.loads(data)
    # for glassDoor API we will be transforming the result into more structured form

    data = extract_salary_data(res, company, location)
    return data



In [69]:
result = glassDoor('microsoft', 'software devloper', 'new york')

In [70]:
result

[{'company': 'microsoft',
  'job_title': 'Software Engineer',
  'location': 'new york',
  'salary_currency': 'USD',
  'salary_period': 'annual',
  'salary_range': 'USD 179,986 - USD 268,933',
  'median_salary': 'USD 218,104',
  'mean_salary': 'N/A',
  'company_rating': 'N/A'},
 {'company': 'microsoft',
  'job_title': 'Product Manager',
  'location': 'new york',
  'salary_currency': 'USD',
  'salary_period': 'annual',
  'salary_range': 'USD 196,016 - USD 286,821',
  'median_salary': 'USD 234,862',
  'mean_salary': 'N/A',
  'company_rating': 'N/A'},
 {'company': 'microsoft',
  'job_title': 'Senior Software Engineer',
  'location': 'new york',
  'salary_currency': 'USD',
  'salary_period': 'annual',
  'salary_range': 'USD 236,293 - USD 343,487',
  'median_salary': 'USD 281,862',
  'mean_salary': 'N/A',
  'company_rating': 'N/A'},
 {'company': 'microsoft',
  'job_title': 'Software Developer',
  'location': 'new york',
  'salary_currency': 'USD',
  'salary_period': 'annual',
  'salary_range

In [81]:
# Job_Salary_Data_API - 50 requests / MO
def job_salary_data_api(company, job_title, location, API_KEY=None):
    res = {'status': 'OK', 'request_id': '92815ef3-ac3d-4a08-b64a-1c76b3a334f0', 'data': [{'location': 'New York City, NY', 'job_title': 'Software Developer', 'company': 'Amazon', 'min_salary': 177650.22, 'max_salary': 276253.84, 'median_salary': 219151.35, 'min_base_salary': 120605.66, 'max_base_salary': 169770.67, 'median_base_salary': 143091.94, 'min_additional_pay': 57044.56, 'max_additional_pay': 106483.17, 'median_additional_pay': 76059.41, 'salary_period': 'YEAR', 'salary_currency': 'USD', 'salary_count': 92, 'confidence': 'CONFIDENT', 'result': {'__typename': 'AggregatedSalaryEstimatesResult', 'basePayStatistics': {'__typename': 'StatisticsResult', 'max': 169770.67, 'mean': 143091.94, 'min': 120605.66, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 103418.91}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 120605.66}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 143091.94}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 169770.67}, {'__typename': 'Percentile', 'ident': 'P90', 'value': 197984.15}]}, 'confidence': 'CONFIDENT', 'currency': {'__typename': 'Currency', 'code': 'USD'}, 'employer': {'__typename': 'Employer', 'counts': {'__typename': 'EmployerCounts', 'globalJobCount': {'__typename': 'EmployerJobCountHolder', 'jobCount': 28589}}, 'id': 6036, 'links': {'__typename': 'EiEmployerLinks', 'jobsUrl': '/Jobs/Amazon-Jobs-E6036.htm'}, 'ratings': {'__typename': 'DefaultEmployerRatings', 'overallRating': 3.6}, 'shortName': 'Amazon', 'squareLogoUrl': 'https://media.glassdoor.com/sql/6036/amazon-squarelogo-1552847650117.png'}, 'jobTitle': {'__typename': 'JobTitle', 'id': 30270, 'text': 'Software Developer'}, 'obscuring': 'NONE', 'payPeriod': 'ANNUAL', 'salariesEmploymentStatus': 'REGULAR', 'salaryCount': 92, 'totalAdditionalPayStatistics': {'__typename': 'StatisticsResult', 'max': 106483.17, 'mean': 76059.41, 'min': 57044.56, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 44114.46}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 57044.56}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 76059.41}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 106483.17}, {'__typename': 'Percentile', 'ident': 'P90', 'value': 139188.72}]}, 'totalPayInsights': None, 'totalPayStatistics': {'__typename': 'StatisticsResult', 'max': 276253.84, 'mean': 219151.35, 'min': 177650.22, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 147533.37}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 177650.22}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 219151.35}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 276253.84}, {'__typename': 'Percentile', 'ident': 'P90', 'value': 337172.87}]}}}]}

    salary_data = res['data'][0]
    return {
        'company': salary_data['company'],
        'job_title': salary_data['job_title'],
        'location': salary_data['location'],
        'salary_currency': salary_data['salary_currency'],
        'salary_period': salary_data['salary_period'],
        'salary_range' : f'{salary_data['salary_currency']}{salary_data['min_salary']} - {salary_data['salary_currency']}{salary_data['max_salary']}',
        'median_salary': f'{salary_data['salary_currency']}{salary_data['median_salary']}',
        'mean_salary': f'{salary_data['salary_currency']}{round(salary_data['result']['totalPayStatistics']['mean'])}',
        'company_rating': salary_data['result']['employer']['ratings']['overallRating']
        # 'confidence': salary_data['confidence'],
        # 'min_salary': salary_data['min_salary'],
        # 'max_salary': salary_data['max_salary'],
        # 'salary_count': salary_data['salary_count'],
    }
result = job_salary_data_api('amazon', 'software devloper', 'new york')
pprint(result)

{'company': 'Amazon',
 'company_rating': 3.6,
 'job_title': 'Software Developer',
 'location': 'New York City, NY',
 'mean_salary': 'USD219151',
 'median_salary': 'USD219151.35',
 'salary_currency': 'USD',
 'salary_period': 'YEAR',
 'salary_range': 'USD177650.22 - USD276253.84'}


# The format to be fllowed :-
```
{
    'company': 'Microsoft',
    'job_title': 'Software Engineer',
    'location': 'New York, NY',
    'salary_currency': 'USD',
    'salary_period': 'annual',
    'salary_range': 'USD 160,000 - 260,000',
    'median_salary': 'USD 210,000',
    'mean_salary': 'USD 218,103',
    'company_rating': 4.2
}
```

In [23]:
def carrer_jet_api(company, job_title, location):
    try:
        url = "http://public.api.careerjet.net/search"

        params = {
            "locale_code": "en_US",
            "keywords": f"{company} {job_title}",
            "location": location,
            "affid": "09a7e99d93482b93310a29944476cd33",  
            "user_ip": "1.2.3.4",        
            "user_agent": "Mozilla/5.0", 
            "pagesize": 10,              
            "page": 1
        }
        response = requests.get(url, params=params)
        data = response.json()
        # if there are no jobs based on the request made
        if "jobs" not in data or not data["jobs"]:
            return {"error": "Job data not available ..."}
        # If we got job data then let us filter it as careerJet does not prefer searching based on company filter
        filtered_jobs = [job for job in data.get("jobs", [])if job.get("company", "").lower() == company.lower()]
        # if no filtered jobs exist
        if not filtered_jobs:
            return {"error": "Job data not available ..."}
        # else replace with the filtered jobs
        data['jobs'] = filtered_jobs
        def extract_job_summary(output_dict):
            salaries = []
            job_list = []
            salary_period = {
                'Y': 'YEAR',
                'A': 'ANNUM',
                'M': 'MONTH',
                'W': 'WEEK',
                'D': 'DAY',
                'H': 'HOUR'
            }
            
            for job in output_dict.get("jobs", []):
                title = job.get("title", "")
                job_list.append(title)
                try:
                    min_salary = float(job.get("salary_min", 0))
                    max_salary = float(job.get("salary_max", 0))

                    if min_salary:
                        salaries.append(min_salary)
                    if max_salary and max_salary != min_salary:
                        salaries.append(max_salary)
                except (ValueError, TypeError):
                    continue
            # if not salaries:
            #     return {
            #         'salary_range': 'N/A',
            #         'base_salary': 'N/A',
            #         'median_salary': 'N/A',
            #         'mean_salary': 'N/A',
            #     }
            if(salaries):
                salaries.sort()
                currency = output_dict.get("salary_currency_code")
                salary_range = f'{currency} {min(salaries):.2f} - {currency} {max(salaries):.2f}'
                base_salary = f'{currency} {min(salaries):.2f}'
                mean_salary = sum(salaries) / len(salaries)
                median_salary = (
                    salaries[len(salaries) // 2]
                    if len(salaries) % 2 == 1
                    else (salaries[len(salaries) // 2 - 1] + salaries[len(salaries) // 2]) / 2
                )
            else:
                salary_range = None
                base_salary = None
                median_salary = None
                mean_salary = None

            job_info = {
                # basic information
                # "job_title": job.get("title", "").split(",")[0],  # I removed extra qualifiers like amazon-ADS
                "company_name": next((p['company'] for p in output_dict.get('jobs', [])), None),
                "job_title": job_list,
                "location": next((p['locations'] for p in output_dict.get("jobs", [])), None),
                # base-salary range
                "base_salary": base_salary,
                "median_salary": f'{currency} {median_salary:.2f}' if median_salary is not None else None,
                "mean_salary": f'{currency} {mean_salary:.2f}' if mean_salary is not None else None,
                "salary_range": salary_range,
                "salary_currency": next((p['salary_currency_code'] for p in output_dict.get("jobs", []) if p.get('salary_currency_code')), None),
                "salary_period": salary_period.get(next((p.get("salary_type") for p in output_dict.get("jobs", [])), None),None),
            }
            return job_info
        job_summary = extract_job_summary(data)
        return job_summary
    except Exception as e:
        return {"error": "Job data not available ..."}

In [24]:
ans = carrer_jet_api('apple', 'software engineer', 'new york')

In [25]:
ans

{'company_name': 'Apple',
 'job_title': ['iOS Engineer, Apple Pay Security',
  'Software Engineer (Data Engineering)',
  'Senior Software Engineer - AppleTV Sports',
  'Software Development Engineer',
  'Software Development Engineer',
  'Software Engineer, Ad Platforms',
  'Senior Software Engineer, Ad Platforms',
  'Software Engineer - ML Data',
  'Senior Software Engineer Frontend, Ad Platforms'],
 'location': 'New York City, NY',
 'base_salary': 'None 151819.00',
 'median_salary': 'None 183159.50',
 'mean_salary': 'None 183159.50',
 'salary_range': 'None 151819.00 - None 214500.00',
 'salary_currency': 'USD',
 'salary_period': None}

In [ ]:
import requests
import urllib.parse
headers = {
    'x-app-id': 'c7740ed4-2dd7-4b3a-b11e-d8c546927444',
    'x-client-id': 'WpSCtxgRVtUWLO1905yPuj1lhu22',
    'x-api-key': 'pEovSWskDx0vzcvbs8EXbO9AFzVlVxPo10gJ1mDoEnM='
}

query_params = {
    
}
url = 'https://gateway.apyflux.com/v1/estimated-salary?job_title=NodeJS Developer&location=New-York, NY, USA&radius=200&fields='
response = requests.get(url, headers=headers, timeout=10)
print(response.text)
